In [1]:
!pip install import_ipynb

In [1]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
import import_ipynb
from IPython.core.interactiveshell import InteractiveShell
import numpy as np
import pandas as pd
import tqdm
import getdata
import generate_features
import model
import pickle
import warnings
warnings.filterwarnings("ignore")

importing Jupyter notebook from getdata.ipynb
importing Jupyter notebook from generate_features.ipynb
importing Jupyter notebook from model.ipynb


In [2]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem.Fingerprints import FingerprintMols

In [3]:
#Do this the first time if needing to download db
data_path = './data/BindingDB'
data_path = getdata.download_BindingDB(data_path)

In [4]:
#Preprocess dataset
df_3d = getdata.process_BindingDB(path = data_path, y = 'Kd')

In [5]:
df_3d.info()

In [6]:
df_3d.to_csv('df_ligand_protein.csv',index=False)

In [7]:
df_3d=pd.read_csv('df_ligand_protein.csv',dtype=str)

In [8]:
df_smiles=df_3d['ligand_smiles']
df_protein=df_3d['bindingdb_target_chain__sequence']
df_ba=df_3d['bindingaffinity']

In [9]:
#run this for the first time to generate ligand features
morgan_matrix_generated=generate_features.generate_morgan_matrix(df_smiles)
morgan_matrix_df=pd.DataFrame(morgan_matrix_generated)
morgan_matrix_df.to_csv('morgan_matrix_generated_ligandprotein.csv',index=False)

In [10]:
morgan_matrix_df=pd.read_csv('morgan_matrix_generated_ligandprotein.csv',dtype=str)

In [11]:
morgan_matrix_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
#dindex

In [13]:
dindex=[1460,4119,4120,4121,4122,4123,4124,4193,4194,4195]

In [14]:
#Drop any values for which Morgan fingerprints were not generated

df_protein.drop(df_protein.index[dindex], axis=0, inplace=True)
df_ba.drop(df_ba.index[dindex], axis=0, inplace=True)


<b>Feature Engineering</b>

In [15]:
#Get feature matrices
test_size=0.2
LX_train,LX_test,Ly_train,Ly_test=generate_features.ligand_features(morgan_matrix_df,df_ba,test_size)
PX_train,PX_test,Py_train,Py_test=generate_features.protein_features(df_protein,df_ba,test_size)

Generating ligand features...
Ligand features after PCA: 191
Ligand features complete!
Generating protein features...
Protein features complete!


In [16]:
print(LX_train.shape),print(PX_train.shape)
print(LX_test.shape),print(PX_test.shape)

(41818, 191)
(41818, 50)
(10455, 191)
(10455, 50)


(None, None)

In [17]:
PX_train_df = pd.DataFrame(PX_train)
LX_train_df = pd.DataFrame(LX_train)

PX_test_df = pd.DataFrame(PX_test)
LX_test_df = pd.DataFrame(LX_test)

In [18]:
PX_train_df.columns = PX_train_df.columns.map(lambda x: str(x) + '_b')
PX_test_df.columns = PX_test_df.columns.map(lambda x: str(x) + '_b')

In [19]:
df_total_train=PX_train_df.join(LX_train_df)
df_total_test=PX_test_df.join(LX_test_df)

y_train=Py_train
y_test=Py_test

In [23]:
#lr_test_pred=model.linear_regression(df_total_train, y_train, df_total_test, y_test)
print('Linear Regression done')

#dt_test_pred=model.decisiontree(df_total_train, y_train, df_total_test, y_test)
print('DT done')

rf_test_pred=model.randomforest(df_total_train, y_train, df_total_test, y_test)
print('RF done')

Linear Regression done
DT done
Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 20.9min
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed: 62.0min finished


Best hyperparameters: {'rf_model__max_features': 'auto', 'rf_model__min_samples_leaf': 10, 'rf_model__n_estimators': 300}
Best model: Pipeline(steps=[('scaler', StandardScaler()),
                ('rf_model',
                 RandomForestRegressor(min_samples_leaf=10, n_estimators=300,
                                       random_state=1))])
Corresponding score: -0.8763649992392982
RF done


In [21]:
#gb_test_pred=model.gradientboosting(df_total_train, y_train, df_total_test, y_test)
#print('GB done')

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 178.4min
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed: 589.4min finished


Best hyperparameters: {'gb_model__learning_rate': 0.01, 'gb_model__max_depth': 10, 'gb_model__n_estimators': 1000}
Best model: Pipeline(steps=[('scaler', StandardScaler()),
                ('gb_model',
                 GradientBoostingRegressor(learning_rate=0.01, max_depth=10,
                                           n_estimators=1000,
                                           random_state=1))])
Corresponding score: -0.8581616630430927
GB done


In [24]:
print(model.score_table)

                          rmse_train rmse_test mae_train  mae_test
LinearRegression             1.11669   1.36256   0.78922   1.04449
DecisionTreeRegressor       0.967801   1.45976  0.656221    1.0269
RandomForestRegressor       0.683945   1.29812  0.444338  0.923616
GradientBoostingRegressor   0.530558   1.28096  0.323515  0.902683


In [ ]:
# Load optimized random forest model

rf_model = pickle.load(open('randomforest_model.sav', 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)